In [1]:
%load_ext autoreload
%autoreload 2

from tqdm import tqdm
import pandas as pd
import numpy as np
import gzip
import json
import math
from datetime import datetime
from IPython.display import display
%matplotlib inline

In [2]:
from helper.manage_orderbooks_v2 import extract_orderbooks_for_one_currencypair, load_orderbook_snapshot
from helper.orderbook_container import OrderbookContainer
from helper.orderbook_trader import OrderbookTradingSimulator

In [3]:
filename = '3000orderbooks'
data = load_orderbook_snapshot(filename, last_line=100)

100%|██████████| 100/100 [00:04<00:00, 20.83it/s]

Loaded 100 orderbooks from file '3000orderbooks'.


In [4]:
orderbook = data[0]
orderbook.plot(range_factor=1.02, outfile="sample.pdf")
print(orderbook)
orderbook2 = data[1]
print(orderbook2)

print("\nCompare two orderbooks:")
diff = orderbook.compare_with(other=orderbook2)

display(diff.head())
print(diff)

Successfully saved 'sample.pdf'
OrderbookContainer from 2016-11-08T10:00
  499 bids (best: 705.0)
  407 asks (best: 705.45)
  kind: 'orderbook'
OrderbookContainer from 2016-11-08T10:01
  499 bids (best: 703.8)
  406 asks (best: 705.45)
  kind: 'orderbook'

Compare two orderbooks:


,Amount,Volume,VolumeAcc,norm_Price,Type
685.11,0.473604,324.471015,108526.174016,0.971477,bid
678.34,0.511551,347.005553,142564.324900,0.961877,bid
674.59,0.546871,368.913775,150950.334759,0.956560,bid
705.45,-1.876972,2237.815171,2237.815171,1.000319,ask
706.89,0.105160,74.336899,2805.184977,1.002361,ask
707.18,6.456261,5649.346841,8454.531818,1.002772,ask


OrderbookContainer from 2016-11-08T10:01
  12 bids (best: 674.59)
  15 asks (best: 705.45)
  kind: 'diff'


In [5]:
vol = 40
print("Current MarketOrderPrice for {} shares: {}".format(vol, orderbook.get_current_price(vol)))

Current MarketOrderPrice for 40 shares: 28358.1256107


In [6]:
display(orderbook.head(2))
display(orderbook.tail(2))

,Amount,Type,Volume,VolumeAcc,norm_Price
703.700000,0.001250,bid,0.879625,112.433157,0.997838
705.000000,0.158232,bid,111.553532,111.553532,0.999681
705.224976,NaN,center,NaN,NaN,NaN
705.450000,3.172181,ask,2237.815171,2237.815171,1.000319
706.170000,0.052838,ask,37.312497,2275.127669,1.001340


,Amount,Type,Volume,VolumeAcc,norm_Price
614.000000,1.838163,bid,1128.632254,326770.989935,0.870644
614.320000,0.034763,bid,21.355643,325642.357681,0.871098
705.224976,NaN,center,NaN,NaN,NaN
810.000000,0.286798,ask,232.306137,292461.937892,1.148570
811.000000,0.000100,ask,0.081100,292462.018992,1.149988


In [7]:
P = 2
ots = OrderbookTradingSimulator(decisionfrequency=P)
vol = 40
T = 10
for e in range(T):
    res = ots.trade_timespan(orderbooks=data[P*e:P*(e+1)], volume=vol, limit=707.5, verbose=True)
    vol = ots.history.volume_left.values[-1]
# res = ots.trade_timespan(orderbooks=data[2:4], volume=40, limit=705.5, timespan=2)

display(ots.history)

Traded 21.6770/40.0000 shares for -15317.2981269, 18.3230 shares left
Traded 1.8090/18.3230 shares for -1276.12574571, 16.5141 shares left
Traded 0.0000/16.5141 shares for 0, 16.5141 shares left
Traded 0.0000/16.5141 shares for -0.00846, 16.5140 shares left
Traded 8.0784/16.5140 shares for -5712.63775969, 8.4356 shares left
Traded 0.0000/8.4356 shares for 0, 8.4356 shares left
Traded 1.5367/8.4356 shares for -1086.67292909, 6.8990 shares left
Traded 6.2908/6.8990 shares for -4448.57031287, 0.6082 shares left
Traded 0.0022/0.6082 shares for -1.527578, 0.6061 shares left
No shares left at t=1 (self.t=20), Done!
Traded 0.6061/0.6061 shares for -428.681746093, 0.0000 shares left


,ASK,BID,CENTER,LIMIT,SPREAD,T,VOLUME,avg,cashflow,cost,cost_avg,high,low,volume_left,volume_traded
2016-11-08T10:00,705.45,705.00,705.224976,707.5,0.45,2,40.000000,706.61535,-15317.298127,2.526130e+01,1.16535,707.50,705.450000,18.323004,21.676996
2016-11-08T10:02,705.45,703.80,704.624678,707.5,1.65,2,18.323004,705.45000,-1276.125746,2.273737e-13,0.00000,705.45,705.450000,16.514051,1.808953
2016-11-08T10:04,707.52,703.80,705.658366,707.5,3.72,2,16.514051,0.00000,0.000000,0.000000e+00,0.00000,0.00,inf,16.514051,0.000000
2016-11-08T10:06,707.52,703.80,705.658366,707.5,3.72,2,16.514051,705.00000,-0.008460,-5.400000e-06,-0.45000,705.00,705.000000,16.514039,0.000012
2016-11-08T10:08,707.52,704.11,705.813627,707.5,3.41,2,16.514039,707.15000,-5712.637760,1.373327e+01,1.70000,707.15,707.150000,8.435643,8.078396
2016-11-08T10:10,707.52,704.11,705.813627,707.5,3.41,2,8.435643,0.00000,0.000000,0.000000e+00,0.00000,0.00,inf,8.435643,0.000000
2016-11-08T10:12,707.52,704.00,705.758537,707.5,3.52,2,8.435643,707.16000,-1086.672929,2.627709e+00,1.71000,707.16,707.160000,6.898971,1.536672
2016-11-08T10:14,707.52,704.00,705.758537,707.5,3.52,2,6.898971,707.16000,-4448.570313,1.075719e+01,1.71000,707.16,707.160000,0.608216,6.290755
2016-11-08T10:16,707.52,704.00,705.758537,707.5,3.52,2,0.608216,707.21204,-1.527578,3.806000e-03,1.76204,707.30,707.100000,0.606056,0.002160
2016-11-08T10:18,707.33,705.00,706.164359,707.5,2.33,2,0.606056,707.33000,-428.681746,1.139386e+00,1.88000,707.33,707.330000,0.000000,0.606056
